In [ ]:
# Importieren der Bibliotheken
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

import plotly.graph_objs as go

# Einlesen der Daten
df = pd.read_csv('/content/GOOGL_historical_data.csv')
df.head()

# Uns interessieren zwei Features: Das Datum und der jeweilige Schlusskurs. Alle weiteren Features können wir erstmal verwerfen.
df['Date'] = pd.to_datetime(df['Date'])
df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)

# Mittels plotly können interaktive Graphiken generiert werden
trace1 = go.Scatter(
    x = df['Date'],
    y = df['Close'],
    mode = 'lines',
    name = 'Data'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1], layout=layout)
fig.show()

# Wir Teilen Schlusskurs und Datum in Trainings- und Testdaten auf
close_data = df['Close'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))


close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Date'][:split]
date_test = df['Date'][split:]

print(len(close_train))
print(len(close_test))

# Wir definieren einen look back von 15 Tagen und nutzen den TimeSeriesGenerator von Keras, um das gewünschte Format zu erhalten
look_back = 15

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

from keras.models import Sequential
from keras.layers import LSTM, Dense

# Initialisiert ein sequenzielles Modell. Alle Schichten werden in einer linearen Reihenfolge gestapelt.
model = Sequential()
# LSTM mit einer Schicht: 10 Neuronen, ReLU Aktivierungsfunktion und Definition des Inputs (Anzahl Zeitschritte, Anzahl Features)
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
# Fügt dem Modell einen Fully Connected Layer hinzu und überführt alle Berechnungen in einen Neuronen zur Vorhersage eines einzelnen kontinuierlichen Werts
model.add(Dense(1))
# Konfiguration von Optimizer und Verlustfunktion
model.compile(optimizer='adam', loss='mse')

num_epochs = 100
model.fit(train_generator, epochs=num_epochs, verbose=1)

# Wir machen Vorhersagen für unsere Testdaten
prediction = model.predict(test_generator)

# Wir überführen unsere daten in ein 1-dimensionales Array, das sie einfacher zu visualisieren sind.
close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

# Es werden Ausgangsdaten, Preditions und Ground Truth visualisiert
trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

4100
1025
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning:

Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.



205/205 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 1049.4794
Epoch 2/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 24.7528
Epoch 3/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 23.2768
Epoch 4/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 5.8322
Epoch 5/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 5.9625
Epoch 6/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 4.6095
Epoch 7/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.8345
Epoch 8/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.4770
Epoch 9/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.5903
Epoch 10/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.5752
Epoch 11/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.4902
Epoch 12/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.6932
Epoch 13/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.4543
Epoch 14/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.4600
Epoch 15/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 